In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np

import mysql.connector
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings('ignore')

from passwords import CURSOR
from passwords import ENGINE

import sys
sys.path.insert(1, '../')

from src import add_tabl